# 신용카드 사기 검출 

일반적인 사기 검출(Fraud Detection)이나 이상 검출(Anomaly Detection)과 같은 데이터 세트는 레이블 값이 극도로 불균형한 분포를 가지기 쉬움.   
그 이유는 사기와 같은 이상 현상은 전체 데이터에서 차지하는 비중이 매우 적을 수 밖에 없음 

### 불균형한 레이블의 데이터 세트
레이블이 불균형한 분포를 가진 데이터 세트를 학습시킬 때 예측 성능 문제 발생 가능성이 존재.  
-> 이는 이상 레이블을 가지는 데이터 건수가 정상 레이블 가진 데이터 건수에 비해 너무 적기 떄문에 발생함.   

이상 레이블을 가지는 데이터 건수는 매우 적기 때문에 제대로 다양한 유형을 학습을 못함.  
반면, 정상 레이블을 가지느 데이터 건수는 매우 많기 때문에 일방적으로 정상 레이블에 치우는 학습을 수행해 제대로 된 이상 데이터 검출이 어려워지기 쉬움.  

이러한 불균형한 레이블 분포로 인한 문제를 해결하기 위해서 적절한 학습 데이터를 확보하는 방안이 필요.  
-> Over Sampling / Under Sampling 

    * Over Sampling : 이상 데이터와 같이 적은 데이터 세트를 증식하여 학습을 위한 충분한 데이터를 확보하는 방법.  
                    단순히 증식하는 방법은 과적합이 되기 때문에 원본 데이터 피처 값들을 아주 약간만 변경하여 증식.  
                    ex) SMOTE(Synthetic Minority Over-Sampling Technique)   
                        : 적은 데이터 세트에 있는 개별 데이터들의 K Nearest Neighbor을 찾아서 이 데이터와 K개 이웃들의 차이를 일정 값으로 만들어서   
                        기존 데이터와 약간 차이가 나는 새로운 데이터들을 생성해 내는 방식. 
                        - from imblearn.over_sampling import SMOTE
                        
    
    * Under Sampling : 많은 데이터 세틑를 적은 데이터 세트 수준으로 감소시키는 방법.  
                    정상 레이블 데이터를 이상 레이블 데이터 수준으로 줄여버리는 방식으로 이 상태에서 학습을 수행할 경우, 정상 레이블 위주의 학습/예측의 부작용을 개선할 수 있지만,  
                    너무 많은 정상 레이블 데이터를 감소함에 따라 제대로 된 학습을 수행할 수 없다는 단점이 있어 잘 적용하지 않는 방법. 


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

card_df = pd.read_csv('data/creditcard.csv')
card_df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [2]:
card_df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [4]:
print('이상 레이블의 비율 : {:2%}'.format(card_df[card_df['Class'] == 1].Class.count()/card_df['Class'].count()) )

이상 레이블의 비율 : 0.172749%


전체 데이터 세트 중 이상 레이블의 비중은 0.17% 밖에 차지하지 않음.

### #1.데이터 일차 가공 및 모델 학습/예측/평가

In [5]:
from sklearn.model_selection import train_test_split 

# 인자로 입력받은 DataFrame을 복사한 뒤 Time 칼럼만 삭제하고 복사된 DataFrame 반환 

def get_preprocessed_df(df = None):
    df_copy = df.copy()
    df_copy.drop('Time', axis =1, inplace =True)
    
    return df_copy

In [8]:
#사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수 

def get_train_test_dataset(df= None):
    # 1. 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환 
    df_copy = get_preprocessed_df(df)
    
    # 2. DataFrame의 맨 마지막 칼럼이 레이블, 나머지 features 
    X_features = df_copy.iloc[:,:-1]
    y_target = df_copy.iloc[:,-1]
    
    # 3. train_test_split()로 데이터 분할 
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size = 0.3,
                                                       random_state = 0, stratify = y_target ) # stratify : 학습과 테스트 데이터 세트의 레이블 분포도 동일하게 지정  
    
    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

In [10]:
print('학습 데이터 레이블 값 비율 ')
print(y_train.value_counts()/y_train.shape[0] * 100)

print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100 )

학습 데이터 레이블 값 비율 
0    99.827451
1     0.172549
Name: Class, dtype: float64
테스트 데이터 레이블 값 비율
0    99.826785
1     0.173215
Name: Class, dtype: float64


#### 로지스틱 회귀 모델 

In [11]:
from sklearn.linear_model import LogisticRegression 

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)

In [12]:
from mlGuideFunc import get_clf_eval 

In [13]:
get_clf_eval(y_test, lr_pred)

오차행렬
[[85279    16]
 [   59    89]]
정확도 : 0.9991 , 정밀도 : 0.8476, 재현율 : 0.6014, F1 : 0.7036, auc : 0.8006


#### LigthGBM 

In [14]:
def get_model_train_eval(model, ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)

In [16]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs =1, boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

오차행렬
[[85290     5]
 [   36   112]]
정확도 : 0.9995 , 정밀도 : 0.9573, 재현율 : 0.7568, F1 : 0.8453, auc : 0.8783
